In [16]:
import os
import random
import soundfile as sf
from tqdm import tqdm
import numpy as np
import csv
from pathlib import Path
import scipy.signal

In [17]:
nearEnd_dir = "D:/near_end_signal/near_end_signal/audio_files"
echoSignal_dir = "D:/AEC-Challenge/datasets/synthetic/echo_signal"
output_dir = "D:/Synthetic_Neural_AEC_dataset/NoiseAndDistortion"
os.makedirs(output_dir, exist_ok=True)
#SNR_range = [10,40]
#SER_range = [-10,10] 

In [18]:
def resample_signal(audio, orig_sr, target_sr):
    """Đổi tần số lấy mẫu của âm thanh."""
    if orig_sr == target_sr:
        return audio
    num_samples = int(len(audio) * float(target_sr) / orig_sr)
    return scipy.signal.resample(audio, num_samples)

In [19]:
def caculate_rms(audio):
    if len(audio)==0:
        return 0
    return np.sqrt(np.mean(audio**2))

def relength_echo_signal(source_audio, target_len):
    target_len = int(target_len)
    currentLength = len(source_audio)

    if currentLength < target_len:
        diff = target_len - currentLength
        if len(source_audio.shape) == 2:
            # Stereo:
            channels = source_audio.shape[1]
            padding = np.zeros((diff, channels))
        else:
            # Mono
            padding = np.zeros(diff)
        return np.concatenate((source_audio, padding))
    elif currentLength > target_len:
        return source_audio[:target_len]
    else:
        return source_audio

"""Optional: Mô phỏng hiện tượng phi tuyến"""
def apply_distortion(audio_sig):
    if random.random()>0.7:
        return audio_sig,"linear"
    distortion_type = 'clipping'
    distorted_sig = audio_sig.copy()

    if distortion_type == 'clipping':
        max_val = np.max(np.abs(audio_sig))
        if max_val > 0:
            threshold = random.uniform(0.8,0.95) * max_val
            distorted_sig = np.clip(audio_sig, -threshold, threshold)
    return distorted_sig, distortion_type

def calculate_gain(clean_rms, noise_rms, target_snr):
    if noise_rms == 0 or clean_rms == 0:
        return 1.0
    target_rms_noise = clean_rms / (10 ** (target_snr / 20))
    return target_rms_noise / noise_rms

def mix_data(nearEnd_dir, echoSignal_dir, output_dir):

    micSignal_dir = os.path.join(output_dir, "mic")
    refSignal_dir = os.path.join(output_dir, "ref")
    Path(micSignal_dir).mkdir(parents=True, exist_ok=True)
    Path(refSignal_dir).mkdir(parents=True, exist_ok=True)
    metadata_path = os.path.join(output_dir, "dataset_config.csv")
    validFileExtensions = ('.wav', '.mp3')
    nearEndFiles = sorted([f for f in os.listdir(nearEnd_dir) if f.lower().endswith(validFileExtensions)])
    echoSignalFiles = [f for f in os.listdir(echoSignal_dir) if f.lower().endswith(validFileExtensions)]
    
    noise_dir = "D:/Downloads/archive/ESC-50-master/audio"
    noiseSignalfiles = [f for f in os.listdir(noise_dir) if f.lower().endswith(validFileExtensions)]
    
    print(f"Processng {len(nearEndFiles)} files...")

    #metadata.csv
    with open(metadata_path, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['id', 'mic_path','ref_path','ser_db', 'snr_db', 'distortion_type', 'source_near_end','source_noise', 'source_echo']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        for ne_filename in tqdm(nearEndFiles, desc="Mixing"):
            try:
                ne_path = os.path.join(nearEnd_dir, ne_filename)
                nearEnd_sig, sr = sf.read(ne_path)

                echo_filename = random.choice(echoSignalFiles)
                echo_path = os.path.join(echoSignal_dir, echo_filename)
                echo_sig, sr_echo = sf.read(echo_path)

                if sr != sr_echo:
                    echo_sig = resample_signal(echo_sig, sr_echo, sr)

                target_len = len(nearEnd_sig)
                echo_aligned = relength_echo_signal(echo_sig, target_len)

                target_ser_db = random.uniform(-5,10)
                rms_ne = caculate_rms(nearEnd_sig)
                rms_echo = caculate_rms(echo_aligned)

                if rms_echo == 0 or rms_ne == 0:
                    echo_scaled = echo_aligned
                else:
                    target_rms_echo = rms_ne / (10**(target_ser_db/20))
                    gain = target_rms_echo / rms_echo
                    echo_scaled = echo_aligned * gain

                has_noise = False
                noise_add = np.zeros_like(nearEnd_sig)
                noise_filename = "None"
                target_snr = 0 
                
                if random.random()>0.5 and len(noiseSignalfiles)>0:
                    has_noise = True
                    noise_filename = random.choice(noiseSignalfiles)
                    noise_path = os.path.join(noise_dir, noise_filename)
                    noise_sig, sr_noise = sf.read(noise_path)
                    
                    if sr != sr_noise:
                        noise_sig = resample_signal(noise_sig, sr_noise, sr)
                    noise_aligned = relength_echo_signal(noise_sig, target_len)  
                    rms_noise = caculate_rms(noise_aligned)                            
                    target_snr = random.uniform(10,40)
                    gain_noise = calculate_gain(rms_ne, rms_noise, target_snr)
                    noise_add = noise_aligned * gain_noise

                echo_distortion, dist_type = apply_distortion(echo_scaled)
                mic_signal = nearEnd_sig + echo_distortion + noise_add
                ref_signal = echo_scaled

                max_amp = np.max(np.abs(mic_signal))
                if max_amp > 1.0:
                    mic_signal /= max_amp
                    ref_signal /= max_amp
                
                out_mic_path = os.path.join(micSignal_dir, ne_filename)
                out_ref_path = os.path.join(refSignal_dir, ne_filename)

                sf.write(out_mic_path, mic_signal, sr)
                sf.write(out_ref_path, ref_signal, sr)

                writer.writerow({
                    'id': os.path.splitext(ne_filename)[0],
                    'mic_path': os.path.join("mic", ne_filename),
                    'ref_path': os.path.join("ref", ne_filename),
                    'ser_db': target_ser_db,
                    'snr_db': target_snr,
                    'distortion_type': dist_type,
                    'source_near_end': ne_filename,
                    'source_noise': noise_filename,
                    'source_echo': echo_filename
                })
            except Exception as e:
                print(f"Error processing {ne_filename}: {e}")            

In [20]:
"""
"""
mix_data(nearEnd_dir,echoSignal_dir,output_dir)

Processng 7991 files...


Mixing: 100%|██████████| 7991/7991 [18:26<00:00,  7.22it/s]
